In [0]:
WITH base AS (

    SELECT
        vendedor.idVendedor,
        pedido.idPedido,
        item_pedido.idProduto,
        item_pedido.vlPreco,
        item_pedido.vlFrete,
        produto.descCategoria,
        produto.nrTamanhoNome,
        produto.nrTamanhoDescricao,
        produto.nrFotos,
        produto.vlComprimentoCm * produto.vlAlturaCm * produto.vlLarguraCm AS produto_volume,
        produto.vlPesoGramas
    FROM silver.olist.vendedor
    LEFT JOIN silver.olist.item_pedido ON vendedor.idVendedor = item_pedido.idVendedor
    LEFT JOIN silver.olist.pedido ON item_pedido.idPedido = pedido.idPedido
    LEFT JOIN silver.olist.produto ON item_pedido.idProduto = produto.idProduto
    WHERE 
        pedido.dtPedido < '2017-06-01'
        AND produto.descCategoria IS NOT NULL

),

medidas AS (

    SELECT
        idVendedor,
        COUNT(DISTINCT idProduto) AS qtd_produtos_unicos,   -- QtdProdutos (ID) únicos (tamanho portfólio)
        COUNT(DISTINCT descCategoria) AS qtd_categorias_distintas,   -- Número de categorias distintas vendidas pelo seller
        AVG(nrTamanhoNome) AS media_tamanho_nome_produto,   -- Média do tamanho do nome do produto
        AVG(nrTamanhoDescricao) AS media_tamanho_nome_descricao,    -- Média do tamanho do nome da descrição
        AVG(nrFotos) AS media_quantidade_fotos,   -- Média da quantidade de fotos
        SUM(CASE WHEN nrFotos IS NOT NULL THEN 1 ELSE 0 END)/COUNT(idProduto) AS taxa_produtos_com_fotos,   -- taxa de produtos com fotos (%)
        SUM(CASE WHEN nrTamanhoDescricao IS NOT NULL THEN 1 ELSE 0 END)/COUNT(idProduto) AS taxa_descricao_produtos,   -- Indicador de se existe descrição atrelada ao anúncio (%)
        AVG(produto_volume) AS media_volume_produtos,   -- Cubagem média dos produtos vendidos (H X L X C)
        AVG(vlPesoGramas) AS media_peso_produtos    -- Peso médio dos produtos vendidos
    FROM base 
    GROUP BY 1

),

base_categoria_mais_vendida AS (

    SELECT
        idVendedor,
        descCategoria,
        COUNT(*) AS quantidade_categoria_principal_mais_vendida
    FROM base
    GROUP BY 1, 2

),

categoria_mais_vendida AS (   -- CategoriaPrincipal (mais vendida)

    SELECT
        idVendedor,
        descCategoria,
        quantidade_categoria_principal_mais_vendida,
        RANK() OVER (PARTITION BY idVendedor ORDER BY quantidade_categoria_principal_mais_vendida DESC) AS rank_categoria_mais_vendida
    FROM base_categoria_mais_vendida
    QUALIFY rank_categoria_mais_vendida = 1

),    -- Avaliar resultado da categoria mais vendida

top_15_categoria_mais_vendida AS (

    SELECT
        idVendedor,
        descCategoria,
        quantidade_categoria_principal_mais_vendida,
        RANK() OVER (PARTITION BY idVendedor ORDER BY quantidade_categoria_principal_mais_vendida DESC) AS rank_categoria_mais_vendida
    FROM base_categoria_mais_vendida
    QUALIFY rank_categoria_mais_vendida <= 15

),

total_categoria_mais_vendida AS (

    SELECT 
        idVendedor,
        SUM(quantidade_categoria_principal_mais_vendida) AS total_categoria_principal_mais_vendida
    FROM top_15_categoria_mais_vendida
    GROUP BY 1

),

share_categoria_mais_vendida AS (
  
    SELECT
        top_15_categoria_mais_vendida.idVendedor,
        descCategoria,
        quantidade_categoria_principal_mais_vendida/total_categoria_principal_mais_vendida
    FROM top_15_categoria_mais_vendida
    LEFT JOIN total_categoria_mais_vendida ON top_15_categoria_mais_vendida.idVendedor = total_categoria_mais_vendida.idVendedor

),

base_categoria_mais_receita AS (

    SELECT
        idVendedor,
        descCategoria,
        SUM(vlPreco) + SUM(vlFrete) AS categoria_principal_maior_receita
    FROM base    
    GROUP BY 1, 2

),

categoria_mais_receita AS (   -- CategoriaPrincipal (maior receita)

    SELECT
        idVendedor,
        descCategoria,
        categoria_principal_maior_receita,
        RANK() OVER (PARTITION BY idVendedor ORDER BY categoria_principal_maior_receita DESC) AS rank_categoria_mais_vendida
    FROM base_categoria_mais_receita
    QUALIFY rank_categoria_mais_vendida = 1

)   -- Avaliar resultado da categoria com maior receita

SELECT * FROM share_categoria_mais_vendida